# 04 - Object Tracking

Module 3: Kalman Filter, Optical Flow (Lucas-Kanade), and DeepSORT.

Knowledge applied:
- Kalman Filter (linear state estimation, predict/update cycle)
- Optical Flow (Lucas-Kanade pyramidal method)
- DeepSORT (Kalman + Hungarian + appearance features)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.insert(0, os.path.abspath('..'))

from src.object_tracking import (
    BallKalmanTracker,
    OpticalFlowEstimator,
    BallTracker,
    DeepSORTTracker,
)
from src.object_detection import Detection

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Kalman Filter - Ball Tracking Demo

Demonstrates the predict/update cycle with simulated ball trajectory.

In [ ]:
# Simulate a parabolic ball trajectory with noise
np.random.seed(42)

# Ground truth trajectory (parabolic)
t = np.linspace(0, 2, 60)  # 2 seconds at 30 fps
gt_x = 100 + 200 * t  # moving right
gt_y = 400 - 300 * t + 200 * t**2  # parabolic arc

# Noisy measurements (simulating detector output)
noise_std = 5.0
meas_x = gt_x + np.random.randn(len(t)) * noise_std
meas_y = gt_y + np.random.randn(len(t)) * noise_std

# Some frames have missing detections
missing_frames = {15, 16, 17, 30, 31, 32, 33}  # indices with no detection

# Run Kalman Filter
tracker = BallKalmanTracker(
    process_noise_std=5.0,
    measurement_noise_std=noise_std,
)

kf_positions = []
kf_predictions = []

for i in range(len(t)):
    if i in missing_frames:
        # No detection: predict only
        pos = tracker.update(None)
    else:
        det = Detection(
            bbox=(int(meas_x[i])-3, int(meas_y[i])-3,
                  int(meas_x[i])+3, int(meas_y[i])+3),
            confidence=0.9,
            class_id=0,
            class_name='ball',
            center=(meas_x[i], meas_y[i]),
        )
        pos = tracker.update(det)
    kf_positions.append(pos)

kf_x = [p[0] for p in kf_positions]
kf_y = [p[1] for p in kf_positions]

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# XY trajectory
axes[0].plot(gt_x, gt_y, 'g-', linewidth=2, label='Ground Truth')
axes[0].scatter(meas_x, meas_y, c='red', s=10, alpha=0.5, label='Noisy Measurements')
axes[0].plot(kf_x, kf_y, 'b-', linewidth=2, label='Kalman Filter')

# Mark missing frames
for i in missing_frames:
    axes[0].plot(kf_x[i], kf_y[i], 'bx', markersize=10)

axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
axes[0].set_title('Ball Trajectory - Kalman Filter')
axes[0].legend()
axes[0].invert_yaxis()

# Position error over time
errors = np.sqrt((np.array(kf_x) - gt_x)**2 + (np.array(kf_y) - gt_y)**2)
meas_errors = np.sqrt((meas_x - gt_x)**2 + (meas_y - gt_y)**2)

axes[1].plot(t, meas_errors, 'r-', alpha=0.5, label='Measurement Error')
axes[1].plot(t, errors, 'b-', label='Kalman Filter Error')
for i in missing_frames:
    axes[1].axvspan(t[i]-0.01, t[i]+0.01, alpha=0.2, color='yellow')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Position Error (pixels)')
axes[1].set_title('Tracking Error (yellow = missing detections)')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'Mean Kalman error: {np.mean(errors):.2f} px')
print(f'Mean measurement error: {np.mean(meas_errors):.2f} px')
print(f'Error reduction: {(1 - np.mean(errors)/np.mean(meas_errors))*100:.1f}%')

## 2. Optical Flow (Lucas-Kanade) Visualization

In [ ]:
# Demonstrate optical flow on synthetic moving object
flow_estimator = OpticalFlowEstimator()

# Create two synthetic frames with a moving bright spot
frame1 = np.zeros((360, 640, 3), dtype=np.uint8)
frame2 = np.zeros((360, 640, 3), dtype=np.uint8)

# Ball at (200, 180) in frame 1, moved to (210, 175) in frame 2
cv2.circle(frame1, (200, 180), 5, (0, 255, 255), -1)
cv2.circle(frame2, (210, 175), 5, (0, 255, 255), -1)

# Compute optical flow
ball_pos = (200.0, 180.0)
vel1 = flow_estimator.estimate_ball_velocity(frame1, ball_pos)
vel2 = flow_estimator.estimate_ball_velocity(frame2, ball_pos)

if vel2 is not None:
    print(f'Optical Flow velocity estimate: vx={vel2[0]:.2f}, vy={vel2[1]:.2f}')
    print(f'Expected: vx=10.0, vy=-5.0')
else:
    print('Optical flow computed (velocity will be available after 2nd frame)')

## 3. Combined Ball Tracker (Kalman + Optical Flow)

In [ ]:
# Combined tracker demonstration
combined_tracker = BallTracker(
    process_noise_std=5.0,
    measurement_noise_std=3.0,
    max_missing_frames=10,
)

# Simulate tracking with a dummy frame
dummy_frame = np.zeros((720, 1280, 3), dtype=np.uint8)

# First few frames with detection
for i in range(5):
    det = Detection(
        bbox=(100+i*10, 200, 110+i*10, 210),
        confidence=0.9, class_id=0, class_name='ball',
        center=(105+i*10, 205),
    )
    pos, track = combined_tracker.update(dummy_frame, det)
    if pos:
        print(f'Frame {i}: pos=({pos[0]:.1f}, {pos[1]:.1f}), det=({det.center[0]:.1f}, {det.center[1]:.1f})')

# Frames without detection (ball occluded)
for i in range(5, 10):
    pos, track = combined_tracker.update(dummy_frame, None)
    if pos:
        print(f'Frame {i}: pos=({pos[0]:.1f}, {pos[1]:.1f}), [PREDICTED - no detection]')

if track:
    print(f'\nTrack ID: {track.track_id}')
    print(f'Total positions: {len(track.positions)}')
    print(f'Active: {track.is_active}')

## 4. DeepSORT Player Tracking

In [ ]:
# DeepSORT tracker demo
player_tracker = DeepSORTTracker(
    max_age=30,
    n_init=3,
)

dummy_frame = np.zeros((720, 1280, 3), dtype=np.uint8)

# Simulate player detections over multiple frames
for frame_i in range(10):
    # Two players moving
    p1 = Detection(
        bbox=(100+frame_i*5, 200, 160+frame_i*5, 400),
        confidence=0.9, class_id=1, class_name='player',
    )
    p2 = Detection(
        bbox=(800-frame_i*3, 250, 860-frame_i*3, 450),
        confidence=0.85, class_id=1, class_name='player',
    )
    
    tracked = player_tracker.update([p1, p2], dummy_frame)
    
    if tracked:
        track_info = ', '.join([f'ID={tid}' for tid, det in tracked])
        print(f'Frame {frame_i}: {len(tracked)} tracked players [{track_info}]')

print(f'\nTotal unique tracks: {len(player_tracker.tracks)}')